In [2]:
from collections import defaultdict

In [3]:
def parse_input(file):
    with open(file) as file_in:
        numbers = file_in.read().splitlines()

    numbers = tuple(map(int, numbers))
    return numbers

In [4]:
def mix(value, secret):
    return value ^ secret

In [5]:
def prune(secret):
    return secret % 16777216

In [6]:
def get_next_secret(secret):
    secret = prune(mix(secret * 64, secret))
    secret = prune(mix(int(secret / 32), secret))
    secret = prune(mix(secret * 2048, secret))
    return secret

In [7]:
def get_nth_secret(secret, n):
    for __ in range(n):
        secret = get_next_secret(secret)
    return secret

In [8]:
def main1(file):
    buyers_secrets = parse_input(file)

    result = 0
    for secret in buyers_secrets:
        result += get_nth_secret(secret, n=2000)

    return result

In [9]:
def extract_price(secret):
    return int(str(secret)[-1])

In [10]:
def get_sequence_changes(secret, n):
    price = extract_price(secret)
    seq = [(price, ' ')]
    for i in range(1, n+1):
        new_secret = get_next_secret(secret)
        new_price = extract_price(new_secret)
        seq.append((new_price, new_price - price))
        secret = new_secret
        price = new_price
    return seq

In [11]:
def main2(file):
    buyers_secrets = parse_input(file)
    buyers_sequences = []

    for secret in buyers_secrets:
        buyers_sequences.append(get_sequence_changes(secret, n=2000))

    letter2mint = {'a': -1, 'b': -2, 'c': -3, 'd': -4, 'e': -5, 'f': -6, 'g': -7, 'h': -8, 'i': -9, ' ': None}
    mint2letter = {v: k for k, v in letter2mint.items()}

    sequences_prices = []
    sequences_diff = []
    for seq in buyers_sequences:
        sequences_prices.append(','.join([str(x[0]) for x in seq]))
        sequences_diff.append(','.join([str(x[1]) if x[1] >= 0 else mint2letter[x[1]] for x in seq[1:]]))

    patterns_freq = defaultdict(dict)

    for i, diff_seq in enumerate(sequences_diff):
        for j in range(0, len(diff_seq), 2):
            pattern = diff_seq[j:j+7]
            if i not in patterns_freq[pattern]:
                patterns_freq[pattern][i] = j
            else:
                continue

    patterns_sum = {pattern: 0 for pattern in patterns_freq}
    for pattern in patterns_freq:
        for seq, idx in patterns_freq[pattern].items():
            if idx < len(sequences_prices[seq]) - 7:
                patterns_sum[pattern] += int(sequences_prices[seq][idx + 8])

    return max(patterns_sum.values())

In [12]:
assert main1('example1.txt') == 37327623

In [13]:
main1('input.txt')

12979353889

In [16]:
assert main2('example2.txt') == 23

In [17]:
main2('input.txt')

1449